In [1]:
import pandas as pd 
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

import seaborn as sns
import matplotlib.pyplot as plt

jan_2022 = pd.read_parquet("/home/ubuntu/mlops-zoomcamp/01-intro/notebooks/data/yellow_tripdata_2022-01.parquet")

In [2]:
jan_2022.shape
print("Q1: ", len(jan_2022.columns))

Q1:  19


In [3]:
jan_2022['duration'] = jan_2022.tpep_dropoff_datetime - jan_2022.tpep_pickup_datetime
jan_2022.duration = jan_2022.duration.apply(lambda td: td.total_seconds() / 60)
print("Q2: ", round(jan_2022['duration'].std(),2))

Q2:  46.45


In [4]:
jan_2022_filtered = jan_2022[(jan_2022.duration >= 1) & (jan_2022.duration <= 60)]
print("Q3: ", round(100*len(jan_2022_filtered)/len(jan_2022)), "%")

Q3:  98 %


In [5]:
categorical = ['PULocationID', 'DOLocationID']

jan_2022_filtered[categorical] = jan_2022_filtered[categorical].astype(str)

/tmp/ipykernel_5469/3555547268.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_2022_filtered[categorical] = jan_2022_filtered[categorical].astype(str)


In [6]:
train_dicts = jan_2022_filtered[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = jan_2022_filtered[target].values


In [7]:
print("Q4: ", X_train.shape)

Q4:  (2421440, 515)


In [8]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_on_train = lr.predict(X_train)

print("Q5: ", round(mean_squared_error(y_train, y_pred_on_train, squared=False),2)) 

Q5:  6.99


In [9]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


In [10]:
feb_2022 = read_dataframe('./data/yellow_tripdata_2022-02.parquet')

In [11]:
target = 'duration'
val_dicts = feb_2022[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
y_val = feb_2022[target].values

In [12]:
y_pred_on_val = lr.predict(X_val)

print("Q6: ", round(mean_squared_error(y_val, y_pred_on_val, squared=False),2))

Q6:  7.79
